<a href="https://colab.research.google.com/github/SahilBeniwal22/4th-year-project/blob/main/Age_Detection_UTK_Dataset_1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile
import os

# Path to the downloaded zip file
zip_path = "/content/archive.zip"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("UTKFace")

print("Dataset extracted successfully.")

Dataset extracted successfully.


In [4]:
import cv2
import numpy as np
import os
from tensorflow.keras.utils import to_categorical

# Dataset path
dataset_path = "UTKFace/UTKFace"

# Image size for model input
IMG_SIZE = 64

# Lists to store images and labels
X = []
y = []

# Read images from the dataset folder
for file in os.listdir(dataset_path):
    if file.endswith(".jpg"):
        try:
            # Extract age from filename
            age = int(file.split("_")[0])

            # Load and preprocess image
            img_path = os.path.join(dataset_path, file)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize to (64,64)
            image = image / 255.0  # Normalize

            X.append(image)
            y.append(age)

        except Exception as e:
            print(f"Error processing {file}: {e}")

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

print(f"Loaded {len(X)} images and labels successfully.")


Loaded 23708 images and labels successfully.


In [5]:

from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


Training samples: 18966, Testing samples: 4742


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D

# Load MobileNetV2 without the top layer
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(64, 64, 3))

# Freeze the base model's layers (optional, can be fine-tuned later)
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Alternative to Flatten() to reduce dimensions
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(1, activation='linear')(x)  # Regression output

# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',  # Regression task
              metrics=['mae'])

model.summary()


<ipython-input-6-3b28037c3772>:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(64, 64, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 64, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 32, 32, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 32, 32, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 32, 32, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 32, 32, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 32, 32, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 32, 32, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 32, 32, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 32, 32, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 32, 32, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 33, 33, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 16, 16, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,430,273 (9.27 MB)

 Trainable params: 172,289 (673.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# Unfreeze some layers of MobileNetV2 for fine-tuning
base_model.trainable = True  # Enable training for base model layers

# Optionally, freeze initial layers to retain learned features
for layer in base_model.layers[:100]:
    layer.trainable = False  # Freeze first 100 layers, train only deeper layers

# Compile the model again
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Reduced LR
              loss='mean_absolute_error',
              metrics=['mae'])


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

# Apply to training set
train_generator = datagen.flow(X_train, y_train, batch_size=32)


In [9]:
history = model.fit(train_generator,
                    validation_data=(X_test, y_test),
                    epochs=30,  # Increase training duration
                    batch_size=32,
                    verbose=1)


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


593/593 ━━━━━━━━━━━━━━━━━━━━ 70s 76ms/step - loss: 18.0494 - mae: 18.0494 - val_loss: 35.8228 - val_mae: 35.8228
Epoch 2/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 30s 51ms/step - loss: 10.4057 - mae: 10.4057 - val_loss: 21.6039 - val_mae: 21.6039
Epoch 3/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - loss: 9.7435 - mae: 9.7435 - val_loss: 15.9919 - val_mae: 15.9919
Epoch 4/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 30s 51ms/step - loss: 9.1167 - mae: 9.1167 - val_loss: 10.1864 - val_mae: 10.1864
Epoch 5/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - loss: 8.8303 - mae: 8.8303 - val_loss: 9.9891 - val_mae: 9.9891
Epoch 6/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - loss: 8.4426 - mae: 8.4426 - val_loss: 12.0814 - val_mae: 12.0814
Epoch 7/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - loss: 8.2984 - mae: 8.2984 - val_loss: 9.2861 - val_mae: 9.2861
Epoch 8/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - loss: 8.1664 - mae: 8.1664 - val_loss: 9.2534 - val_mae: 9.2534
Epoch 9/30
593/593 ━━━━━━━━━━━━━━━━━━

In [11]:
from sklearn.metrics import mean_absolute_error, confusion_matrix


# Predictions on the test set
y_pred = model.predict(X_test)

# Calculate MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Updated Mean Absolute Error: {mae:.2f}")

# Convert predictions to integers
y_pred_int = y_pred.round().astype(int)

# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_int)
print("Updated Confusion Matrix:\n", conf_matrix)


149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step
Updated Mean Absolute Error: 6.62
Updated Confusion Matrix:
 [[92 82 20 ...  0  0  0]
 [26 32 18 ...  0  0  0]
 [ 3  8 13 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [12]:
model.save('age_detection_model.h5')